## Spam_Detector

In [9]:
%time

import os
import re
import string
import math
 
DATA_DIR = '/Users/punchh/Documents/my_work/Spam_Detector/enron'
target_names = ['ham', 'spam']
 
def get_data(DATA_DIR):
    subfolders = ['enron%d' % i for i in range(1,7)]
 
    data = []
    target = []
    for subfolder in subfolders:
        # spam
        spam_files = os.listdir(os.path.join(DATA_DIR, subfolder, 'spam'))
        for spam_file in spam_files:
            with open(os.path.join(DATA_DIR, subfolder, 'spam', spam_file), encoding="latin-1") as f:
                data.append(f.read())
                target.append(1)
 
        # ham
        ham_files = os.listdir(os.path.join(DATA_DIR, subfolder, 'ham'))
        for ham_file in ham_files:
            with open(os.path.join(DATA_DIR, subfolder, 'ham', ham_file), encoding="latin-1") as f:
                data.append(f.read())
                target.append(0)
 
    return data, target


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


In [10]:
# This will produce two lists: the data list, where each element is the text of an email, and the target list, 
# which is simply binary (1 meaning spam and 0 meaning ham).

In [42]:
class SpamDetector(object):
    """Implementation of Naive Bayes for binary classification"""
    def clean(self, s): # function to clean up our string by removing punctuation.
        
        translator = str.maketrans("", "", string.punctuation)
        """ This uses the 3-argument version of str.maketrans
            with arguments (x, y, z) where 'x' and 'y'
            must be equal-length strings and characters in 'x'
            are replaced by characters in 'y'. 'z'
            is a string (string.punctuation here)
            where each character in the string is mapped
            to None """
        return s.translate(translator)
 
    def tokenize(self, text): # function to tokenize our string into words.
        text = self.clean(text).lower()
        return re.split("\W+", text)
 
    def get_word_counts(self, words): # function to count up how many of each word appears in a list of words.
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts
    
    
    
    def fit(self, X, Y):
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()
 
        n = len(X)
        self.num_messages['spam'] = sum(1 for label in Y if label == 1)
        self.num_messages['ham'] = sum(1 for label in Y if label == 0)
        self.log_class_priors['spam'] = math.log(self.num_messages['spam'] / n)
        self.log_class_priors['ham'] = math.log(self.num_messages['ham'] / n)
        self.word_counts['spam'] = {}
        self.word_counts['ham'] = {}
 
        for x, y in zip(X, Y):
            """zip() to map the similar index of multiple containers 
                so that they can be used just using as single entity."""
        
            c = 'spam' if y == 1 else 'ham'
            counts = self.get_word_counts(self.tokenize(x))
            for word, count in counts.items():
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0
 
                self.word_counts[c][word] += count
    
    
    
    
    def predict(self, X):
        result = []
        for x in X:
            counts = self.get_word_counts(self.tokenize(x))
            spam_score = 0
            ham_score = 0
            for word, _ in counts.items():
                if word not in self.vocab: continue
            
                # adding Laplace smoothing
                log_w_given_spam = math.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )
                log_w_given_ham = math.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )
 
                spam_score += log_w_given_spam
                ham_score += log_w_given_ham
 
            spam_score += self.log_class_priors['spam']
            ham_score += self.log_class_priors['ham']
 
            if spam_score > ham_score:
                result.append(1)
            else:
                result.append(0)
        return result

Sudo-code for the algorithm:
    1. Compute log class priors by counting how many messages are spam/ham, 
       dividing by the total number of messages, and taking the log.
    2. For each (document, label) pair, tokenize the document into words.
    3. For each word, either add it to the vocabulary for spam/ham, 
       if it isn’t already there, and update the number of counts. 
       Also add that word to the global vocabulary.

Now that we’ve extracted all of the data we need from the training data, 
we can write another function to actually output the class label for new data. 
To do this classification, we apply Naive Bayes directly. 

For example, given a document, we need to iterate each of the words 
and compute log p(w_i|{Spam}) and sum them all up, 
and we also compute log p(w_i|{Ham}) and sum them all up. 
Then we add the log class priors and check to see which score is bigger for that document. 
Whichever is larger, that is the predicted label!

NOTE: remember that the log of 0 is undefined! What if we encounter a word that is in the “spam” vocabulary, but not the “ham” vocabulary? Then p(w_i|{Ham}) will be 0! One way around this is to use `Laplace Smoothing`. We simply add 1 to the numerator, but we also have to add the size of the vocabulary to the denominator to balance it.

In [43]:
"""def predict(self, X):
    result = []
    for x in X:
        counts = self.get_word_counts(self.tokenize(x))
        spam_score = 0
        ham_score = 0
        for word, _ in counts.items():
            if word not in self.vocab: continue
            
            # adding Laplace smoothing
            log_w_given_spam = math.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )
            log_w_given_ham = math.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )
 
            spam_score += log_w_given_spam
            ham_score += log_w_given_ham
 
        spam_score += self.log_class_priors['spam']
        ham_score += self.log_class_priors['ham']
 
        if spam_score > ham_score:
            result.append(1)
        else:
            result.append(0)
    return result"""

"def predict(self, X):\n    result = []\n    for x in X:\n        counts = self.get_word_counts(self.tokenize(x))\n        spam_score = 0\n        ham_score = 0\n        for word, _ in counts.items():\n            if word not in self.vocab: continue\n            \n            # adding Laplace smoothing\n            log_w_given_spam = math.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )\n            log_w_given_ham = math.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )\n \n            spam_score += log_w_given_spam\n            ham_score += log_w_given_ham\n \n        spam_score += self.log_class_priors['spam']\n        ham_score += self.log_class_priors['ham']\n \n        if spam_score > ham_score:\n            result.append(1)\n        else:\n            result.append(0)\n    return result"

In [44]:
# input can be a list of document texts; we return a list of predictions

In [47]:
"""We’re reserving the first 100 for the testing set, “train”
    our Naive Bayes classifier, then compute the accuracy."""

if __name__ == '__main__':
    X, y = get_data(DATA_DIR)
    MNB = SpamDetector()
    MNB.fit(X[100:], y[100:])
 
    pred = MNB.predict(X[:100])
    true = y[:100]
 
    accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
    print("{0:.3f}%".format(accuracy*100))

94.000%
